# Download representative images of volcanos


Credit: this script derived from [Aaron Geller's blog post.](https://sites.northwestern.edu/researchcomputing/2021/11/19/downloading-satellite-images-made-easy/), including the use of functions based on his [EarthEngineToGeoTIFF repository.](https://github.com/ageller/EarthEngineToGeoTIFF)

In [ ]:
import ee
%pip install rasterio

import rasterio
from rasterio.plot import show as showRasterio

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta
import zipfile
import os
import requests

import numpy as np


In [ ]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
TMP_DIR = PROJECT_DIR + "TempDIR/"
OUTPUT_IMAGE_DIR = PROJECT_DIR +"Full_Sentinel_sat_10km/"



targets = pd.read_excel(PROJECT_DIR + "Recent Eruptions.xlsx")



Mounted at /content/drive


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# Sentinel 2 cloud mask removal from https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

# percent cloud filter allowable for sentinel2 images
CLOUD_FILTER = 70
CLD_PRB_THRESH = 20
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
      # Get s2cloudless image, subset the probability band.
      cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

      # Condition s2cloudless by the probability threshold value.
      is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

      # Add the cloud probability layer and cloud mask as image bands.
      return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


In [ ]:
def getBestImage(lon, lat, sze_km, rgb_filename, nir_filename,
                         dateMin = '2020-04-01', dateMax = '2020-04-30', 
                         min = 0, vmax = 3500,
                         temp_dir = TMP_DIR):
    '''    
    download an RGB image from the Sentinal S2 Surface Reflectance satellite, at the given coordinates
    
    lon : central longitude in degrees
    lat : central latitude in degrees
    sze_km : size of the edge of the box in km
    dateMin : minimum date to use for image search in year-month-day (e.g., 2020-08-01)
    dateMax : maximum date to use for image search in year-month-day (e.g., 2020-08-31)
    vMin : minimum value to select in the Sentinal image pixels (I think this should be close to 0)
    vMax : maximum value to select in the Sentinal image pixels (I think this should be close to 3000)
    rgb_filename : output filename for the RGB GeoTIFF image
    nir_filename : output filename for the RGB GeoTIFF image
    
    Note: it's possible that the vMin and vMax values should be different for each band to make the image look nicer
    
    https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    '''


    print('Downloading Sentinel S2 Surface Reflectance satellite images ... ')
    
    # define the area of interest, using the Earth Engines geometry object

    pt = ee.Geometry.Point(lon,lat)
    
    aoi = ee.Geometry.buffer(pt,sze_km)

    # get sentinal image remove clouds
    col = get_s2_sr_cld_col(aoi, dateMin, dateMax)\
                            .map(add_cld_shdw_mask)\
                            .map(apply_cld_shdw_mask)

    db = ee.Image(col.mosaic())

    # add the latitude and longitude
    db = db.addBands(ee.Image.pixelLonLat())


    # define the bands that I want to use.  B4 is red, B3 is green, B2 is blue, b8 is near IR, 
    # https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands
    bands = ['B4', 'B3', 'B2','B5', 'B9', 'B8']

    # export geotiff images, these go to Drive and then are downloaded locally
    for selection in bands:
        task = ee.batch.Export.image.toDrive(image=db.select(selection),
                                     description=selection,
                                     scale=30,
                                     region=aoi,
                                     fileNamePrefix=selection,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
        task.start()

        url = db.select(selection).getDownloadURL({
            'scale': 30,
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
            'region': aoi})
    
        r = requests.get(url, stream=True)

        filenameZip = temp_dir + selection+'.zip'
        filenameTif = selection+'.tif'

        # unzip and write the tif file, then remove the original zip file
        with open(filenameZip, "wb") as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)

        zipdata = zipfile.ZipFile(filenameZip)
        zipinfos = zipdata.infolist()

        # iterate through each file (there should be only one)
        for zipinfo in zipinfos:
            zipinfo.filename = filenameTif
            zipdata.extract(zipinfo,path=temp_dir)
    
        zipdata.close()
        
    # create a combined RGB geotiff image
    # https://gis.stackexchange.com/questions/341809/merging-sentinel-2-rgb-bands-with-rasterio
    print('Creating RGB image ... ')
    B2 = rasterio.open(temp_dir +'B2.tif')
    B3 = rasterio.open(temp_dir +'B3.tif')
    B4 = rasterio.open(temp_dir +'B4.tif')

    # get the scaling
    image = np.array([B2.read(1), B3.read(1), B4.read(1)]).transpose(1,2,0)
    p2, p98 = np.percentile(image, (2,98))

    # use the B3 image as a starting point so that I keep the same parameters
    B3_geo = B3.profile
    B3_geo.update({'count': 3})

    with rasterio.open(rgb_filename, 'w', **B3_geo) as dest:
        dest.write( (np.clip(B4.read(1), p2, p98) - p2)/(p98 - p2)*255, 1)
        dest.write( (np.clip(B3.read(1), p2, p98) - p2)/(p98 - p2)*255, 2)
        dest.write( (np.clip(B2.read(1), p2, p98) - p2)/(p98 - p2)*255, 3)


    dest.close()

    print('Creating NIR image ... ')
    # open the images
    B5 = rasterio.open(temp_dir +'B5.tif')
    B8 = rasterio.open(temp_dir +'B8.tif')
    B9 = rasterio.open(temp_dir +'B9.tif')

    # get the scaling
    image = np.array([B5.read(1), B8.read(1), B9.read(1)]).transpose(1,2,0)
    p2, p98 = np.percentile(image, (2,98))

    with rasterio.open(nir_filename, 'w', **B3_geo) as dest:
        dest.write( (np.clip(B5.read(1), p2, p98) - p2)/(p98 - p2)*255, 1)
        dest.write( (np.clip(B8.read(1), p2, p98) - p2)/(p98 - p2)*255, 2)
        dest.write( (np.clip(B9.read(1), p2, p98) - p2)/(p98 - p2)*255, 3)


    dest.close()
    
    B5.close()
    B8.close()
    B9.close()
    
    # remove the intermediate files
    for selection in bands:
        os.remove(temp_dir +selection + '.tif')
        os.remove(temp_dir +selection + '.zip')
  


In [ ]:
def get_image_data(longitude, latitude, buffer_size,
                             output_rgb,output_grn, 
                            dateMin , dateMax ):
   # download stacked tif files
    _ = getBestImage(longitude, latitude, buffer_size,
                             output_rgb+".tif",output_grn+".tif", 
                            dateMin , dateMax )

    #convert from tif to png
    for file in [output_rgb,output_grn]:
        f,ax = plt.subplots(figsize=(15,15))

        img = rasterio.open(file+".tif")

        showRasterio(img.read(), ax = ax, transform=img.transform)

        f.savefig(file+".png", bbox_inches='tight')

        img.close()

        plt.close(f)
        os.remove(file + ".tif")

    


In [ ]:
target_size_km = 10000
for i_eruption in range(84,targets.shape[0]):
    volc = targets.loc[i_eruption]
    v_name = re.sub(r"[,.]","",re.sub(r"\s+", "_",volc["Volcano Name"]))
   

    # adjust eruption date by 1 year
    erupt_start_date = date(volc["Start Year"].astype(int),
                            volc["Start Month"].astype(int),
                            volc["Start Day"].astype(int))
    erupt_end_date = date(volc["End Year"].astype(int),
                          volc["End Month"].astype(int),
                          volc["End Day"].astype(int))

    if (erupt_start_date >= date(2015,6,23)):
      print(v_name,erupt_start_date)
    # do prior to eruption too
      pre_erupt_date = erupt_start_date - timedelta(weeks=1)

      volc_file_name = f'{OUTPUT_IMAGE_DIR}{v_name}__{pre_erupt_date}_pre_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
      print(i_eruption, volc_file_name)
        
      try:
          get_image_data(volc["Longitude"], volc["Latitude"], target_size_km,
                              volc_file_name,volc_file_name+"_b5_b8_b9", 
                              dateMin = pre_erupt_date.isoformat(), dateMax=erupt_start_date.isoformat())
      except Exception as e:
          print(f'Failed to get data for {volc_file_name}')
          print(e)
      
      next_start_date = erupt_start_date
      next_end_date = erupt_start_date + timedelta(days=7)
      while (next_end_date <= erupt_end_date + timedelta(days=7) and next_end_date <= erupt_start_date + timedelta(days=187)):
          volc_file_name = f'{OUTPUT_IMAGE_DIR}{v_name}_{next_start_date}_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'

          print(i_eruption, volc_file_name)
          try:
              get_image_data(volc["Longitude"], volc["Latitude"], target_size_km,
                              volc_file_name,volc_file_name+"_b5_b8_b9", 
                              dateMin = next_start_date.isoformat(), dateMax=next_end_date.isoformat())
          except Exception as e:
            print(f'Failed to get data for {volc_file_name}:\n {e}')
          next_start_date = next_start_date + timedelta(days=7)
          next_end_date = next_end_date + timedelta(days=7)
        